In [5]:

from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [6]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
# pc.create_index(
#     name = "rag", dimension = 1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region ="us-east-1")
# )

In [8]:
import json
data = json.load(open('reviews.json'))
data['boba_shops']

[{'shopName': 'Bubble Bliss',
  'location': {'address': '123 Boba St',
   'city': 'Milk Tea City',
   'state': 'CA',
   'country': 'USA',
   'postalCode': '90210'},
  'hours': {'monday': '10:00 AM - 9:00 PM',
   'tuesday': '10:00 AM - 9:00 PM',
   'wednesday': '10:00 AM - 9:00 PM',
   'thursday': '10:00 AM - 9:00 PM',
   'friday': '10:00 AM - 10:00 PM',
   'saturday': '10:00 AM - 10:00 PM',
   'sunday': '11:00 AM - 8:00 PM'},
  'ratings': {'avgRating': 4.7,
   'totalReviews': 128,
   'reviewDetails': [{'rating': 5,
     'comment': "Best taro milk tea I've ever had!",
     'reviewDate': '2024-07-21',
     'reviewer': 'Alice Brown'},
    {'rating': 4,
     'comment': 'Great drinks but a bit crowded on weekends.',
     'reviewDate': '2024-06-15',
     'reviewer': 'John Smith'}]}},
 {'shopName': 'TeaMoo Delight',
  'location': {'address': '456 Pearl Ln',
   'city': 'Boba Town',
   'state': 'TX',
   'country': 'USA',
   'postalCode': '75001'},
  'hours': {'monday': '10:00 AM - 8:00 PM',
   

In [9]:
processed_data = []
client = OpenAI()

for shop in data['boba_shops']:
    for review in shop['ratings']['reviewDetails']:
        response = client.embeddings.create(
            input=review['comment'],
            model='text-embedding-3-small',
        )
        embedding = response.data[0].embedding
        processed_data.append({
            "values": embedding,
            "id": shop["shopName"],
            "metadata": {
                "review": review["comment"],
                "reviewDate": review["reviewDate"],
                "reviewer": review["reviewer"],
                "stars": review["rating"]
            }
        })


In [10]:
processed_data[0]

{'values': [-0.05033333,
  -0.048129275,
  -0.04844414,
  -0.006291666,
  -0.019487858,
  -0.0054539014,
  0.015237189,
  0.057440266,
  -0.014674931,
  -0.019544084,
  0.019791478,
  -0.017801085,
  0.015540808,
  -0.00025038046,
  0.015293415,
  -0.009800155,
  -0.020702336,
  -0.031149087,
  -0.010502977,
  0.03535478,
  0.032588467,
  0.0008736082,
  0.024784328,
  0.033825435,
  0.0024879912,
  0.002892817,
  -0.011110216,
  -0.013258041,
  -0.0151134925,
  -0.0105648255,
  -0.011874887,
  -0.020522414,
  0.020443697,
  -0.04495814,
  0.012785745,
  -0.00294342,
  0.018666962,
  0.012414655,
  -0.030316945,
  0.00747803,
  0.0074667847,
  -0.004829795,
  0.02132082,
  -0.012605822,
  0.037423886,
  -0.007410559,
  -0.026088767,
  0.042191833,
  0.014753647,
  -0.019825213,
  -0.015563299,
  0.012830725,
  -0.063872494,
  -0.009176048,
  -0.030069552,
  -0.027550638,
  -0.023772264,
  0.05046827,
  0.08955644,
  -0.009822645,
  0.022793936,
  0.02057864,
  -0.0074948976,
  -0.03771

In [11]:
index = pc.Index('cowlitea')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 46}

In [12]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 23}},
 'total_vector_count': 23}